## TODO
Convert this notebook to a script and deploy to docker so it can be a cron job

In [3]:
import pandas as pd 
import requests

In [4]:
# https://curlconverter.com/

headers = {
    'authority': 'consumer-api.leafly.com',
    'accept': 'application/json',
    'accept-language': 'en-GB,en;q=0.6',
    # Requests sorts cookies= alphabetically
    'if-none-match': 'W/"224a3f637e080602bd933b9d99864385"',
    'origin': 'https://www.leafly.com',
    'referer': 'https://www.leafly.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36',
    'x-app': 'web-web',
    'x-environment': 'prod',
}



In [5]:
response = requests.get('https://consumer-api.leafly.com/api/strain_playlists/v2?&skip=0&take=60')
num_strains = response.json()['totalCount']


{'hits': {'strain': [{'id': 233839,
    'averageRating': 4.564077669902913,
    'category': 'Hybrid',
    'flowerImageSvg': 'https://public.leafly.com/strains/flowers/wedding-cake-flower.svg',
    'name': 'Wedding Cake',
    'nugImage': 'https://leafly-public.s3-us-west-2.amazonaws.com/strains/photos/m2y50HYRBu0dHY4JSdSx_wedding-cake_jman.jpg',
    'phenotype': 'Hybrid',
    'reviewCount': 2060,
    'shortDescriptionPlain': None,
    'slug': 'wedding-cake',
    'strainTopTerp': 'limonene',
    'subtitle': 'aka Triangle Mints #23, Pink Cookies',
    'thc': 24.0,
    'topEffect': 'Relaxed',
    'cannabinoids': {'cbc': {'displayName': 'CBC',
      'order': 4,
      'percentile25': None,
      'percentile50': 0.0,
      'percentile75': None},
     'cbd': {'displayName': 'CBD',
      'order': 3,
      'percentile25': 0.0,
      'percentile50': 0.0,
      'percentile75': 0.0},
     'cbg': {'displayName': 'CBG',
      'order': 2,
      'percentile25': None,
      'percentile50': 1.0,
      'p

In [6]:
get_batch = lambda x: f'https://consumer-api.leafly.com/api/strain_playlists/v2?&skip={x*60}&take=60&lat=45.6684&lon=-111.2422'

In [ ]:
import math

In [7]:
strains_data = []
for i in range(math.ceil(num_strains/60)):
    try:
        data = requests.get(get_batch(i), headers=headers).json()
        strains_data.extend(iter(data['hits']['strain']))
    except Exception as e:
        print(f'failed to grab 60 at {60*i}')
        print(e)

len(strains_data)

6153

In [8]:
# https://stackoverflow.com/a/41801708
strains_df = pd.concat([pd.json_normalize(v, sep='_') for v in strains_data])
strains_df.reset_index(drop=True, inplace=True)

In [10]:
strains_df.head()

,id,averageRating,category,flowerImageSvg,name,nugImage,phenotype,reviewCount,shortDescriptionPlain,slug,...,terps_myrcene_score,terps_ocimene_name,terps_ocimene_description,terps_ocimene_score,terps_pinene_name,terps_pinene_description,terps_pinene_score,terps_terpinolene_name,terps_terpinolene_description,terps_terpinolene_score
0,233839,4.564078,Hybrid,https://public.leafly.com/strains/flowers/wedd...,Wedding Cake,https://leafly-public.s3-us-west-2.amazonaws.c...,Hybrid,2060,None,wedding-cake,...,0.355683,ocimene,None,0.008820,pinene,None,0.130472,terpinolene,None,0.027740
1,65591,4.639808,Hybrid,https://public.leafly.com/strains/flowers/orig...,GG4,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,4989,Original Glue (GG4) is a potent hybrid strain ...,original-glue,...,0.410000,ocimene,None,0.015000,pinene,None,0.118333,terpinolene,None,0.025000
2,504256,4.557951,Hybrid,https://public.leafly.com/strains/flowers/runt...,Runtz,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,742,None,runtz,...,0.062000,ocimene,None,0.017500,pinene,None,0.091542,terpinolene,None,0.024583
3,503179,4.618434,Indica,https://public.leafly.com/strains/flowers/ice-...,Ice Cream Cake,https://images.leafly.com/flower-images/defaul...,Indica,1009,None,ice-cream-cake,...,0.199178,ocimene,None,0.012209,pinene,None,0.126561,terpinolene,None,0.023545
4,174127,4.600363,Hybrid,https://public.leafly.com/strains/flowers/gela...,Gelato,https://s3-us-west-2.amazonaws.com/leafly-imag...,Hybrid,2202,None,gelato,...,0.128459,ocimene,None,0.027078,pinene,None,0.127167,terpinolene,None,0.016123


In [9]:
strains_df.to_csv('../../data/raw/strains.csv',index=False)


In [ ]:
# https://realpython.com/beautiful-soup-web-scraper-python/#step-3-parse-html-code-with-beautiful-soup